In [1]:
import pandas as pd
import Levenshtein as lv
import numpy as np

In [2]:
first = 'APRLPQCQGDDQEKCLCNKEDCAPGQCRFPRGDADPYCE'
second = 'APRLPQCQGDDQEKCLCNKDECAPGQCRFPRGDADPYCE'
lv.distance(first, second)

2

In [3]:
proteins = pd.read_csv("traincsv.csv")

In [4]:
proteins = proteins.dropna(axis = 0)

proteins

,seq_id,protein_sequence,pH,data_source,tm,Size
0,30277.0,RPAVT,7.0,doi.org/10.1038/s41592-020-0801-4,50.4,5.0
1,30284.0,RPGSI,7.0,doi.org/10.1038/s41592-020-0801-4,49.2,5.0
2,2806.0,LVVGTALNYINIF,7.0,doi.org/10.1038/s41592-020-0801-4,58.3,13.0
3,30387.0,SCQGTGKDSHSFSS,7.0,doi.org/10.1038/s41592-020-0801-4,37.7,14.0
4,1497.0,FGEDIKHLGNSISRAA,7.0,doi.org/10.1038/s41592-020-0801-4,64.4,16.0
...,...,...,...,...,...,...
23582,23904.0,MSGLSNKRAAGDGGSGPPEKKMNREEKTTTTLIEPIRLGGISSTEE...,7.0,doi.org/10.1038/s41592-020-0801-4,45.3,1000.0
23583,26441.0,MSVPAIAPRRKRLADGLSVTQKVFVRSRNGGATKIVREHYLRSDIP...,7.0,doi.org/10.1038/s41592-020-0801-4,48.8,1000.0
23584,26554.0,MSYDYHQSWSRDGGPRGSGQGSSGGGGGGSRGSGGGGGGRGGRGRH...,7.0,doi.org/10.1038/s41592-020-0801-4,52.4,1000.0
23585,29125.0,MVSKMIIENFEALKSWLSKTLEPICDADPSALAKYVLALVKKDKSE...,7.0,doi.org/10.1038/s41592-020-0801-4,48.1,1000.0


In [5]:
proteins['group_number'] = np.nan

proteins

,seq_id,protein_sequence,pH,data_source,tm,Size,group_number
0,30277.0,RPAVT,7.0,doi.org/10.1038/s41592-020-0801-4,50.4,5.0,NaN
1,30284.0,RPGSI,7.0,doi.org/10.1038/s41592-020-0801-4,49.2,5.0,NaN
2,2806.0,LVVGTALNYINIF,7.0,doi.org/10.1038/s41592-020-0801-4,58.3,13.0,NaN
3,30387.0,SCQGTGKDSHSFSS,7.0,doi.org/10.1038/s41592-020-0801-4,37.7,14.0,NaN
4,1497.0,FGEDIKHLGNSISRAA,7.0,doi.org/10.1038/s41592-020-0801-4,64.4,16.0,NaN
...,...,...,...,...,...,...,...
23582,23904.0,MSGLSNKRAAGDGGSGPPEKKMNREEKTTTTLIEPIRLGGISSTEE...,7.0,doi.org/10.1038/s41592-020-0801-4,45.3,1000.0,NaN
23583,26441.0,MSVPAIAPRRKRLADGLSVTQKVFVRSRNGGATKIVREHYLRSDIP...,7.0,doi.org/10.1038/s41592-020-0801-4,48.8,1000.0,NaN
23584,26554.0,MSYDYHQSWSRDGGPRGSGQGSSGGGGGGSRGSGGGGGGRGGRGRH...,7.0,doi.org/10.1038/s41592-020-0801-4,52.4,1000.0,NaN
23585,29125.0,MVSKMIIENFEALKSWLSKTLEPICDADPSALAKYVLALVKKDKSE...,7.0,doi.org/10.1038/s41592-020-0801-4,48.1,1000.0,NaN


In [16]:
proteins.loc[1070]

seq_id                                                         8897.0
protein_sequence    MEFFQLSRKIGCNITQSPIYLFLARKTNRCLCPAHTDFILAVVGPT...
pH                                                                7.0
data_source                         doi.org/10.1038/s41592-020-0801-4
tm                                                               55.9
Size                                                            106.0
group_number                                                      NaN
Name: 1070, dtype: object

In [1]:
#function that takes in a row and applies the levishtein algorithm around it (the ones similar in size) to see if there are any matches that are similar based on a threshold
# if no matches, we drop the data point; if there are matches, we assign them to the same group number.


#your iloc and loc have ruined everything. they are not matched and so they are not working. your group numbers are being assigned wrongly causing your algorithm to make wrong decisions.


def remove_distant(dataframe, primary_index):
    
    
    comparative_index = 0
    
    forwards_direction = 1 # whether we want to check the proteins behind us or in fronmt of us in the table.
    backwards_direction = -1
    
    
    primary_protein = dataframe.loc[primary_index]
    primary_protein_size = dataframe.loc[primary_index, 'Size']
    primary_protein_gn = dataframe.loc[primary_index, 'group_number']
    primary_protein_seq = dataframe.loc[primary_index, 'protein_sequence']

    primary_gn_is_null = pd.isna(primary_protein_gn)
    
    
    #function checks to see any other proteins that are close to this one
    
    def matcher_finder(direction):

        comparative_index = primary_index + direction
        
        difference_in_size = 0
        
        threshold = 5 #levenshtein number threshold

        while (difference_in_size <= 1) and (comparative_index < dataframe.shape[0]):
            
            #print(difference_in_size)
            
            #print(comparative_index, '\n')

            comparative_protein_seq = dataframe.loc[comparative_index, 'protein_sequence']
            
            

            distance = lv.distance(primary_protein_seq, comparative_protein_seq)

            if (distance <= threshold): 
                print('Got one at:', comparative_index,'\n')
                
                #print(primary_protein_seq, '\n')
                
                #print(comparative_protein_seq, '\n')
                
                return comparative_index

            comparative_protein_size = dataframe.loc[comparative_index, 'Size']
            
            comparative_index = comparative_index + direction
            
            difference_in_size =  abs(comparative_protein_size - primary_protein_size)
            
            

        return False 

    

    
    comparative_index = matcher_finder(forwards_direction)
    
    if comparative_index != False:
        comparative_protein_gn = dataframe.loc[comparative_index, 'group_number']
        comparative_gn_is_null = pd.isna(comparative_protein_gn)
        
        if primary_gn_is_null and comparative_gn_is_null ==  False: #primary is null but the comp is not.
                #print('prime is nully\n') 
                primary_protein_gn = comparative_protein_gn

        elif primary_gn_is_null == False and comparative_gn_is_null: #primary is already assigned but the comp is null
            #print('comp is nully\n')
            comparative_protein_gn = primary_protein_gn

        elif primary_gn_is_null and comparative_gn_is_null : #both are null
            #print('both nullies\n')
            
            primary_protein_gn = primary_index
            comparative_protein_gn = primary_protein_gn
            
        #this needs to be included, I made some decisions early to make pandas happy (loc instead of iloc) and to make things clear. This is the assignment. 
        dataframe.loc[primary_index, 'group_number'] =  primary_protein_gn 
    
        dataframe.loc[comparative_index, 'group_number'] =  comparative_protein_gn
    
    elif (comparative_index == False) and primary_gn_is_null == True:
        dataframe.drop(primary_index, inplace=True)
        dataframe.reset_index(drop=True, inplace=True)
        
        #print('we just deleted dis boi\n')
        return True, dataframe
    
    return False, dataframe
        

    

In [11]:
testy = proteins.copy()

In [12]:
testy.shape

(23359, 7)

In [13]:
testy.iloc[range(23,30)] 

,seq_id,protein_sequence,pH,data_source,tm,Size,group_number
23,426.0,APRLPQCQGDDQEKCLCNKDECAPGQCRFPRGDADPYCE,7.0,10.1093/protein/gzi054,66.5,39.0,NaN
24,427.0,APRLPQCQGDDQEKCLCNKDECPAGQCRFPRGDADPYCE,7.0,10.1093/protein/gzi054,66.5,39.0,NaN
25,8462.0,MEAATRRRQLLGAAGGPGVAFVQARQGLYYIVLAVLELH,7.0,doi.org/10.1038/s41592-020-0801-4,53.4,39.0,NaN
26,1616.0,FVFQTVFDTAITSWYENHNKGKLWKDVKARIAAGDGDDDD,7.0,doi.org/10.1038/s41592-020-0801-4,51.6,40.0,NaN
27,9614.0,MEPPNLYPVKLYVYDLSKGLARRLSPIMLGRDIAWAPRLC,7.0,doi.org/10.1038/s41592-020-0801-4,46.4,40.0,NaN
28,21848.0,MRIGPHLAQAPEEAEEGLGPEVVRALEGRIVVEPWGRKPL,7.0,doi.org/10.1038/s41592-020-0801-4,86.7,40.0,NaN
29,27648.0,MTRGNQRELARQKNMKKQSDSVKGKRRDDGLSAAARKQRVG,7.0,doi.org/10.1038/s41592-020-0801-4,48.8,41.0,NaN


In [14]:
#run the frame through the remove distant function


for index in range(23, testy.shape[0]): 
    if (index%500 == 0):
        print('index: ', index, '/23359 \n')
    
    try:

        cont = True


        #we keep removing stuff on the same index level
        while cont == True:
            cont, testy = remove_distant(testy, index)
            
            
    except IndexError:
        pass
    

index:  23 /23359 

Got one at: 24 

index:  24 /23359 

index:  25 /23359 

Got one at: 26 

index:  26 /23359 

index:  27 /23359 

Got one at: 28 

index:  28 /23359 

Got one at: 29 

index:  29 /23359 

Got one at: 30 

index:  30 /23359 

Got one at: 31 

index:  31 /23359 

Got one at: 32 

index:  32 /23359 

Got one at: 33 

index:  33 /23359 

Got one at: 34 

index:  34 /23359 

Got one at: 35 

index:  35 /23359 

Got one at: 36 

index:  36 /23359 

Got one at: 37 

index:  37 /23359 

Got one at: 38 

index:  38 /23359 

Got one at: 39 

index:  39 /23359 

Got one at: 40 

index:  40 /23359 

Got one at: 41 

index:  41 /23359 

Got one at: 42 

index:  42 /23359 

Got one at: 43 

index:  43 /23359 

Got one at: 44 

index:  44 /23359 

Got one at: 45 

index:  45 /23359 

index:  46 /23359 

Got one at: 47 

index:  47 /23359 

Got one at: 48 

index:  48 /23359 

Got one at: 49 

index:  49 /23359 

Got one at: 50 

index:  50 /23359 

Got one at: 51 

index:  51 /233

KeyError: 1070

In [17]:
testy.shape

(1070, 7)

In [19]:
testy

,seq_id,protein_sequence,pH,data_source,tm,Size,group_number
0,30277.0,RPAVT,7.0,doi.org/10.1038/s41592-020-0801-4,50.4,5.0,NaN
1,30284.0,RPGSI,7.0,doi.org/10.1038/s41592-020-0801-4,49.2,5.0,NaN
2,2806.0,LVVGTALNYINIF,7.0,doi.org/10.1038/s41592-020-0801-4,58.3,13.0,NaN
3,30387.0,SCQGTGKDSHSFSS,7.0,doi.org/10.1038/s41592-020-0801-4,37.7,14.0,NaN
4,1497.0,FGEDIKHLGNSISRAA,7.0,doi.org/10.1038/s41592-020-0801-4,64.4,16.0,NaN
...,...,...,...,...,...,...,...
1065,31168.0,VYEDQVGKFDWRQQYVGKIKFASLEFSPGSKKLVVATEKNVIAALN...,7.0,doi.org/10.1038/s41592-020-0801-4,51.3,974.0,1064.0
1066,7572.0,MDGFAGSLDDSISAASTSDVQDRLSALESRVQQQEDEITVLKAALA...,7.0,doi.org/10.1038/s41592-020-0801-4,53.4,987.0,1066.0
1067,7573.0,MDGFAGSLDDSISAASTSDVQDRLSALESRVQQQEDEITVLKAALA...,7.0,doi.org/10.1038/s41592-020-0801-4,51.3,987.0,1066.0
1068,3865.0,MADRKANGGGASASSSGTNLLFSSSATEFSFNVPFIPVTQAAASPA...,7.0,doi.org/10.1038/s41592-020-0801-4,50.2,997.0,1068.0


In [20]:


testy.to_csv('filteredtrain.csv', index=False)

